# Introduction

This notebook will demonstrate how to perform a sentimental analysis using different Deep Learning techniques. As a beginner with Deep Learning Techniques for NLP, I won't use the most sophisticated techniques available out there. However, I'll explain the basics needed to face an NLP project. 

The data that I will be working with in this notebook will be from a Women's Clothing E-Commerce store revolving around the reviews written by customers on their different products. If you want to know more about it, you can click here.

## Acknowledgment 

I want to recognize MR_KNOWNOTHING for his fantastic contribution to NLP's topic in Kaggle's community. Here are a couple of his kernels that I recommend to everyone to check out:

* https://www.kaggle.com/tanulsingh077/deep-learning-for-nlp-zero-to-transformers-bert

* https://www.kaggle.com/tanulsingh077/twitter-sentiment-extaction-analysis-eda-and-model

## About this notebook

For this project, I'll show the performance of the Recurrent Neural Networks (RNN) and Long Short Term Memory networks (LSTM's) techniques for NPL. Previous to that, I'll display a simple EDA to understand the basics of the dataset and preprocess it for the models.

In [1]:
# Importing necessary libraries for preprocessing & EDA
import numpy as np 
import pandas as pd 
from plotly import graph_objs as go

# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

df = pd.read_csv('../input/womens-ecommerce-clothing-reviews/Womens Clothing E-Commerce Reviews.csv')
df.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [2]:
# Remove columns
del df['Unnamed: 0']

# How many reviews do we have?
print('There are', df.shape[0], 'reviews in this dataset')

# Do we have duplicates?
print('Number of Duplicates:', len(df[df.duplicated()]))

# Do we have missing values?
print('Number of Missing Values:', df.isnull().sum().sum())

There are 23486 reviews in this dataset
Number of Duplicates: 21
Number of Missing Values: 4697


In [3]:
print('Number of Missing Values per column:')
df.isnull().sum().sort_values(ascending=False)

Number of Missing Values per column:


Title                      3810
Review Text                 845
Division Name                14
Department Name              14
Class Name                   14
Clothing ID                   0
Age                           0
Rating                        0
Recommended IND               0
Positive Feedback Count       0
dtype: int64

I'll remove the rows with nulls of every column except the <code>Title</code> feature.

In [4]:
# Remove rows with nulls in specific columns
df = df.dropna(subset = ['Review Text', 'Division Name', 'Department Name', 'Class Name'])

# EDA

In [5]:
recommended = (
    df['Recommended IND']
    .value_counts()
    .to_frame()
    .reset_index()
    .rename(columns={'index':'Recommended', 'Recommended IND':'Count'})
    .sort_values(by=['Recommended'], ascending=True)
    .replace([0, 1], ['No', 'Yes'])   
          )  

colors = ['#f6b220','#0E2F44']

fig = go.Figure(data=[go.Pie(labels=recommended['Recommended'], 
                             values=recommended['Count'])])

fig.update_traces(hoverinfo='percent', 
                  textinfo='label', 
                  textfont_size=20,
                  marker=dict(colors=colors, 
                              line=dict(color='white', width=1)))

fig.update_layout(showlegend=False, 
                  title_text="<b>Recommended</b> Distribution",
                  title_x=0.5,
                  font=dict(family="Rockwell, sans-serif", size=25, color='#000000'))

fig.show()

The majority of the reviews are positive and recommend the product they purchase.

In [6]:
classes = (
    df
    .groupby(['Recommended IND', 'Class Name'])
    .size()
    .to_frame()
    .rename(columns={0:'Count'})
    .reset_index()
          )  

# get proportions in each class
a = classes.groupby('Class Name')['Count'].transform('sum')

classes['Count'] = classes['Count'].div(a)

# pivot table
classes = classes.pivot(index='Class Name', columns='Recommended IND')  

fig = go.Figure()
fig.add_trace(go.Bar(
    y=classes.index,
    x=classes.iloc[:,0],
    name='Not Recommended',
    orientation='h',
    marker=dict(
        color='#f6b220')
    ))

fig.add_trace(go.Bar(
    y=classes.index,
    x=classes.iloc[:,1],
    name='Recommended',
    orientation='h',
    marker=dict(
        color='#0E2F44')
    ))

fig.update_layout(barmode='stack')

fig.update_layout(
                title = 'Distribution of <b>Product Class</b> by Recommendation ',
                barmode='stack', 
                autosize=False,
                width=680,
                height=800,
                font=dict(family="Rockwell, sans-serif", size=18, color='#000000'),
                margin=dict(
                  l=150,
                  r=100,
                   b=30,
                   t=100,
                   pad=4
                          ))
fig.layout.xaxis.tickformat = ',.0%'

fig.show()

"Trend" is the class with the most "Not Recommended" responses.

In [7]:
rating = (
    df['Rating']
    .value_counts()
    .to_frame()
    .reset_index()
    .rename(columns={'index':'Rating', 'Rating':'Count'})
    .sort_values(by=['Rating'], ascending=True)   
          ) 

rating['percent'] = ((rating['Count'] / rating['Count'].sum())*100).round(2).astype(str) + '%'

colors = ['#0E2F44',] * 5
colors[4] = '#f6b220'

fig = go.Figure(go.Bar(
            y=rating['Count'],
            x=rating['Rating'],
            marker_color=colors,
            text=rating['percent']
                        ))

fig.update_traces(texttemplate='%{text}', 
                  textposition='outside',
                  cliponaxis = False,
                  hovertemplate='<b>Rating</b>: %{x}<br><extra></extra>'+  # <extra></extra> removes trance 0
                                '<b>Count</b>: %{y}',
                  textfont_size=18)
                  
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
 
fig.update_layout(coloraxis=dict(colorscale='Teal'),
                  showlegend=False, 
                  plot_bgcolor='#F7F7F7', 
                  margin=dict(pad=20),
                  paper_bgcolor='#F7F7F7',
                  height=500,
                  yaxis={'showticklabels': False},
                  yaxis_title=None,
                  xaxis_title=None,
                  title_text="<b>Rating</b> Distribution",
                  title_x=0.5,
                  font=dict(family="Rockwell, sans-serif", size=22, color='#000000'),
                  title_font_size=35)


fig.show()

# NLP

For the following models, I'll just be focusing on the review text and recommendation columns to determine if, based on the text, the user will recommend the product to other customers (i.e. if they like it enough to recommend it to some else). This way it will reduce the complexity of the model and make it a classification binary model.

In [8]:
# Necessary libraries
from sklearn.model_selection import train_test_split
from sklearn import metrics

import re
import string

from tensorflow import keras
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import SimpleRNN, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [9]:
# Let's just work with the reviews and recommendations
data = df[['Review Text', 'Recommended IND']]

In [10]:
# Cleaning the text
def clean_text(text):
    '''Make text lowercase, remove text in square brackets, 
    remove links, remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [11]:
data['Review Text'] = data['Review Text'].apply(lambda x:clean_text(x))

In [12]:
# Setting up the evaluation metrics
def roc_auc(predictions,target):
    
    fpr, tpr, thresholds = metrics.roc_curve(target, predictions)
    roc_auc = metrics.auc(fpr, tpr)
    return roc_auc

In [13]:
# Split target & features
X = data.drop('Recommended IND', axis=1)
y = data['Recommended IND']

# Spliting train & test
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    stratify=y,
                                                    test_size=0.2, 
                                                    random_state=42,
                                                    shuffle=True)

# Tokenization

Tokenization is an essencial step on a NLP process. Tokenization is essentially splitting a phrase, sentence, paragraph, or an entire text document into smaller units, such as individual words or terms. Each of these smaller units are called tokens. Check out the below image to visualize this definition:

<center><img src='https://freecontent.manning.com/wp-content/uploads/Chollet_DLfT_01.png'></center>

I recommend checking these 2 articles: 

* https://www.analyticsvidhya.com/blog/2019/07/how-get-started-nlp-6-unique-ways-perform-tokenization/

* https://www.kdnuggets.com/2020/03/tensorflow-keras-tokenization-text-data-prep.html

In [14]:
# Keras takenization text data prep

num_words = None   # the most X frequent words is returned

# Tokenize data
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(X_train['Review Text'].tolist() + X_test['Review Text'].tolist())   # introduce text in list

# Get data word index
word_index = tokenizer.word_index

# Encode training/test data sentences into sequences
X_train_seq = tokenizer.texts_to_sequences(X_train['Review Text'].tolist())
X_test_seq = tokenizer.texts_to_sequences(X_test['Review Text'].tolist())

# Get max training sequence length
max_len = max([len(x) for x in X_train_seq])

# Pad the training/test sequences
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

# Output some results 
print("\nPadded training shape:", X_train_pad.shape)
print("\nPadded test shape:", X_test_pad.shape)


Padded training shape: (18102, 112)

Padded test shape: (4526, 112)


# RNN

In [15]:
# Basic RNN 
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                    50,     # embeds it in a 50-dimensional vector
                    input_length=max_len))
model.add(SimpleRNN(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 112, 50)           1022500   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 100)               15100     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 1,037,701
Trainable params: 1,037,701
Non-trainable params: 0
_________________________________________________________________



User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_DUPLICATE_LIB_OK=True
   KMP_INIT_AT_FORK=FALSE
   KMP_SETTINGS=1
   KMP_WARNINGS=0

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=true
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_PLAIN_BARRIER='2,2'
   KMP_PLAIN_BARRIER_P

In [16]:
batch_size = 512

model.fit(X_train_pad, y_train, epochs=5, batch_size=batch_size)

2021-11-22 23:59:04.057263: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5
36/36 [==============================] - 8s 179ms/step - loss: 0.4909 - accuracy: 0.8037
Epoch 2/5
36/36 [==============================] - 6s 178ms/step - loss: 0.4540 - accuracy: 0.8186
Epoch 3/5
36/36 [==============================] - 6s 180ms/step - loss: 0.4194 - accuracy: 0.8241
Epoch 4/5
36/36 [==============================] - 7s 206ms/step - loss: 0.4685 - accuracy: 0.7890
Epoch 5/5
36/36 [==============================] - 7s 185ms/step - loss: 0.3560 - accuracy: 0.8452


In [17]:
scores = model.predict(X_test_pad)
print("AUC: %.2f%%" % (roc_auc(scores,y_test)))

AUC: 0.71%


# LSTM's

In [18]:
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                    50,     # embeds it in a 50-dimensional vector
                    input_length=max_len))

model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 112, 50)           1022500   
_________________________________________________________________
lstm (LSTM)                  (None, 100)               60400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 1,083,001
Trainable params: 1,083,001
Non-trainable params: 0
_________________________________________________________________


In [19]:
batch_size = 512

model.fit(X_train_pad, y_train, epochs=5, batch_size=batch_size)

Epoch 1/5
36/36 [==============================] - 44s 1s/step - loss: 0.5233 - accuracy: 0.8107
Epoch 2/5
36/36 [==============================] - 39s 1s/step - loss: 0.3653 - accuracy: 0.8315
Epoch 3/5
36/36 [==============================] - 40s 1s/step - loss: 0.2431 - accuracy: 0.8944
Epoch 4/5
36/36 [==============================] - 39s 1s/step - loss: 0.2009 - accuracy: 0.9173
Epoch 5/5
36/36 [==============================] - 38s 1s/step - loss: 0.1708 - accuracy: 0.9301


In [20]:
scores = model.predict(X_test_pad)
print("AUC: %.2f%%" % (roc_auc(scores,y_test)))

AUC: 0.92%
